In [74]:
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from src.utils import topic_clustering as TopicClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [20]:
data = pd.read_parquet("data/news_processed.parquet")
tf    = load_npz('models/tf_unigram.npz')
tfidf = load_npz('models/tfidf_unigram.npz')
vocabulary = pd.read_csv("models/vocabulary_unigram.csv").ngrams
doc_top_wals = np.load(f'models/doc_topics_wals_unigram.npy').astype('float32')
word_top_wals = np.load(f'models/word_topics_wals_unigram.npy').astype('float32')

In [29]:
words_df = pd.DataFrame({"word":vocabulary.values, 'freq': np.asarray(tf.sum(axis=0))[0]}
                       ).sort_values('freq', ascending=False)

In [67]:
data.sample(2)

,id,html,title,author,href,citation,date,filename,filesize,filetype,filehref
187164,356017-2,titulo como superar impasse nuclear iraniano a...,ira triplicara processamento uranio,None,/bdsf/handle/id/356017,"O Estado de São Paulo, 09/06/2011, Internacion...",09/06/2011,complemento_1.htm,6.999Kb,HTML,/bdsf/bitstream/handle/id/356017/complemento_1...
202907,465999-1,titulo setor eletrico teme herdar passivo r b...,setor eletrico teme herdar passivo r bi,"Rittner, Daniel",/bdsf/handle/id/465999,"Valor Econômico, 13/09/2011, Brasil, p. A7",13/09/2011,noticia.htm,6.626Kb,HTML,/bdsf/bitstream/handle/id/465999/noticia.htm?s...


In [71]:
data.author.value_counts().head(30)

Chade, Jamil              2511
Leitão, Míriam            2305
Pereira, Merval           2208
Macedo, Fausto            1611
Brígido, Carolina         1512
Moreira, Assis            1448
Rittner, Daniel           1364
Marin, Denise Chrispim    1350
Carvalho, Jailton de      1349
Rosa, Vera                1243
Jungblut, Cristiane       1239
Basile, Juliano           1195
Monteiro, Tânia           1172
Domingos, João            1093
Doca, Geralda             1060
Éboli, Evandro            1039
Gallucci, Mariângela      1019
Weber, Demétrio           1013
Netto, Andrei              977
Leo, Sergio                973
Vasconcelos, Adriana       959
Pereira, Renée             947
Kramer, Dora               942
Lopes, Eugênia             937
Ribeiro, Alex              933
Lima, Maria                916
Camarotti, Gerson          890
Beck, Martha               881
Arruda, Roldão             866
Ulhôa, Raquel              862
Name: author, dtype: int64

# Analise

In [45]:
citation_list = data.citation.fillna('').apply(lambda x: x.split(","))

In [46]:
citation_df = pd.DataFrame({"citation": data.citation, "citation_list": citation_list, 
                            "citation_list_lenght": citation_list.apply(lambda x: len(x))})

In [60]:
jornal_counts = citation_df.query("citation_list_lenght==4").citation_list.apply(lambda x: x[0]).value_counts()
caderno_counts = citation_df.query("citation_list_lenght==4").citation_list.apply(lambda x: x[2]).value_counts()

In [72]:
jornal_counts

O Estado de São Paulo       108289
O Globo                      73449
Valor Econômico              51043
O globo                      13366
Jornal O Globo                 226
Jornal O Valor Econômico       116
O Estado de S. Paulo            54
Estado de São Paulo             47
Valor econômico                 16
Folha de S. Paulo               11
Folha de São Paulo               7
​O globo                         2
Name: citation_list, dtype: int64

In [73]:
caderno_counts.head(60)

 Economia                    42148
 Nacional                    35617
 O País                      34605
 Brasil                      12826
 Internacional               12554
 Política                    10377
 Opinião                      7347
 Economia & Negócios          5856
 O Mundo                      5179
 Finanças                     4925
 Vida&                        4037
 Empresas                     3839
 Rio                          3567
 Especial                     3303
 Agronegócios                 2974
 Vida                         2942
 Metrópole                    2589
 Espaço Aberto                2065
 País                         1953
 Notas e Informações          1951
 Notas e informações          1678
 Empresas &                   1546
 Legislação & Tributos        1431
 Espaço aberto                1329
 Negócios                      980
 Notas & Informações           917
 Vida &                        877
 O Globo                       787
 Legislação         

In [57]:
citation_df.query("citation_list_lenght==5")

,citation,citation_list,citation_list_lenght
202,"O Globo, n. 29.767, País, p. 8, fev. 2015.","[O Globo, n. 29.767, País, p. 8, fev. 2015.]",5
283,"O globo, nº 29.734, 03/01/2015, País, p. 7.","[O globo, nº 29.734, 03/01/2015, País, p. 7.]",5
380,"O globo, nº 29.735, 04/01/2015, Economia, p. 26","[O globo, nº 29.735, 04/01/2015, Economia, ...",5
544,"O globo, nº 29.749, 18/01/2015, Economia, p.35.","[O globo, nº 29.749, 18/01/2015, Economia, ...",5
956,"O Globo, n. 29.766, País, p. 9, fev. 2015.","[O Globo, n. 29.766, País, p. 9, fev. 2015.]",5
...,...,...,...
255970,"Valor econômico, v. 17, n. 4057, 28/07/2016. P...","[Valor econômico, v. 17, n. 4057, 28/07/201...",5
255981,"Valor econômico, v. 16, n. 3811, 03/08/2015. E...","[Valor econômico, v. 16, n. 3811, 03/08/201...",5
256005,"Valor econômico, v. 17, n. 4229, 05/04/2017. O...","[Valor econômico, v. 17, n. 4229, 05/04/201...",5
256010,"Valor econômico, v. 15, n. 3745, 29/04/2015. B...","[Valor econômico, v. 15, n. 3745, 29/04/201...",5


In [41]:
a.apply(lambda x:x[]).value_counts()

4    246626
5      7513
3      1760
6        67
7        26
1        19
8         8
Name: citation, dtype: int64

In [34]:
data

,id,html,title,author,href,citation,date,filename,filesize,filetype,filehref
0,406557-2,titulo contribuicao previdencia deve mudar aut...,governo pretende reativar camaras setoriais,"Oliveira, Eliane; Barbosa, Flávia",/bdsf/handle/id/406557,"O Globo, 18/05/2007, Economia, p. 27",18/05/2007,complemento_1.htm,2.784Kb,HTML,/bdsf/bitstream/handle/id/406557/complemento_1...
1,399177-2,titulo cpi podera ouvir de novo exministro aut...,palocci pode indiciado,"Jungblut, Cristiane; Carvalho, Jailton de; Web...",/bdsf/handle/id/399177,"O Globo, 03/04/2006, O País, p. 3",03/04/2006,complemento_1.htm,3.820Kb,HTML,/bdsf/bitstream/handle/id/399177/complemento_1...
2,400292-6,titulo copa mundo meio fumaca autor deborah be...,cerco fumantes europa,"Berlinck, Deborah",/bdsf/handle/id/400292,"O Globo, 21/05/2006, O Mundo, p. 41",21/05/2006,complemento_5.htm,1.916Kb,HTML,/bdsf/bitstream/handle/id/400292/complemento_5...
3,396275-1,titulo virar jogo autor miriam leitao fonte o ...,virar jogo,"Leitão, Míriam",/bdsf/handle/id/396275,"O Globo, 19/01/2006, Economia, p. 22",19/01/2006,noticia.htm,4.388Kb,HTML,/bdsf/bitstream/handle/id/396275/noticia.htm?s...
4,414658-1,titulo o general papa autor casado jose fonte ...,o general papa,"Casado, José",/bdsf/handle/id/414658,"O Globo, 02/03/2008, O País, p. 16",02/03/2008,noticia.htm,4.858Kb,HTML,/bdsf/bitstream/handle/id/414658/noticia.htm?s...
...,...,...,...,...,...,...,...,...,...,...,...
256014,468193-1,titulo copa serve pretexto governo modernizar ...,copa serve pretexto governo modernizar portos,"Rittner, Daniel",/bdsf/handle/id/468193,"Valor Econômico, 03/10/2011, Brasil, p. A4",03/10/2011,noticia.htm,5.864Kb,HTML,/bdsf/bitstream/handle/id/468193/noticia.htm?s...
256015,484288-1,titulo a pirataria marcas era internet autor p...,a pirataria marcas era internet,"Pimenta, Luiz Edgard M.; Espíndola, Marcelle F.",/bdsf/handle/id/484288,"Valor Econômico, 25/01/2008, Legislação, p. E2",25/01/2008,noticia.htm,6.974Kb,HTML,/bdsf/bitstream/handle/id/484288/noticia.htm?s...
256016,466221-1,titulo cresce disputa iss leasing autor fernan...,cresce disputa iss leasing,"Teixeira, Fernando",/bdsf/handle/id/466221,"Valor Econômico, 07/04/2006, Legislação & Trib...",07/04/2006,noticia.htm,4.115Kb,HTML,/bdsf/bitstream/handle/id/466221/noticia.htm?s...
256017,457238-1,titulo frete escoar soja mato grosso forte alt...,frete escoar soja mato grosso forte alta,"Bouças, Cibelle; Lopes, Fernando",/bdsf/handle/id/457238,"Valor Econômico, 27/01/2006, Agronegócios, p. B12",27/01/2006,noticia.htm,4.872Kb,HTML,/bdsf/bitstream/handle/id/457238/noticia.htm?s...


In [30]:
words_df

,word,freq
4589,ano,317663
10616,brasil,316547
26305,ele,253773
4633,anos,252802
45306,lula,243132
...,...,...
24155,digeridos,15
71500,tratarseia,15
25268,dobrariam,15
24741,disponibilizem,15


# Topicos

In [77]:
%%time
topwords_relevance_df_wals_topic, top5_words_relevance_wals_topic = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_topic, top5_words_weight_wals_topic       = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=False, n_top_words=50)

CPU times: user 3.09 s, sys: 59.7 ms, total: 3.15 s
Wall time: 3.15 s


In [78]:
top5_words_relevance_wals_topic

0       sistema | liminar | programas | vagas | programa
1                  semana | passada | mp | acao | senado
2           valores | relacoes | afirma | politico | ser
3      quilometros | arruda | iniciativa | operacao | km
4                dar | navios | tres | marinha | nuclear
                             ...                        
295             henrique | bc | cpi | dossie | meirelles
296             crivella | feita | tv | prb | russomanno
297             nova | horas | primeira | hora | jornada
298               ira | semana | cargos | russia | cinco
299       principal | predio | paulista | serra | evitar
Length: 300, dtype: object

In [81]:
df_words5_topics = pd.DataFrame({"topic_words_weight": top5_words_weight_wals_topic,"topic_words_relevance": top5_words_relevance_wals_topic})

In [84]:
df_words5_topics.iloc[120:180]

,topic_words_weight,topic_words_relevance
120,primeiro | resultado | ficou | direito | votos,resultado | ficou | direito | primeiro | votos
121,proximos | projecoes | previsao | projecao | i...,projecoes | projecao | proximos | previsao | e...
122,nada | faz | forma | associacao | sergio,nada | emendas | associacao | verbas | faz
123,problema | empresas | mas | setores | quer,cursos | alunos | escolas | setores | ensino
124,minas | tambem | poderia | aecio | diante,minas | aecio | neves | ata | gerais
125,relator | cunha | tecnologia | plenario | camara,cunha | relator | tecnologia | dirceu | sarney
126,participacao | secretario | social | reforma |...,participacao | jobim | previdencia | secretari...
127,bem | construcao | ato | ministerio | civil,atos | ato | construcao | bem | acidente
128,essa | forma | ha | zona | eduardo,zona | essa | recorde | lopes | norte
129,menos | comunicacao | dizer | tv | ministro,comunicacao | expressao | midia | tv | dizer


# Clusters

In [85]:
clustering_outputs_wals_200  = np.load('models/clusters_wals_unigram_200_outputs.npz')
data['cluster_wals_200'] = clustering_outputs_wals_200['index']

In [87]:
%%time
word_cluster_wals_200 = TopicClustering.get_word_cluster_matrix(data.cluster_wals_200, tf).astype('float32')
topwords_relevance_df_wals_200, top5_words_relevance_wals_200 = TopicClustering.view_topics(word_cluster_wals_200, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_200, top5_words_weight_wals_200       = TopicClustering.view_topics(word_cluster_wals_200, vocabulary, relevance=False, n_top_words=50)

CPU times: user 2.8 s, sys: 84 ms, total: 2.88 s
Wall time: 2.88 s


In [88]:
df_words5_clusters = pd.DataFrame({"cluster_words_weight": top5_words_weight_wals_200, "cluster_words_relevance": top5_words_relevance_wals_200})

In [90]:
df_words5_clusters.iloc[:60]

,cluster_words_weight,cluster_words_relevance
0,temer | federal | delacao | janot | ministro,joesley | janot | funaro | delacao | jbs
1,cpi | oposicao | senado | pt | lula,cpi | cpis | requerimento | oposicao | assinat...
2,stf | ministro | supremo | tribunal | justica,sabatina | stf | indicacao | supremo | triplice
3,previdencia | inss | fundo | pensao | fundos,previdencia | inss | funpresp | pensao | compl...
4,paises | emissoes | brasil | energia | clima,emissoes | carbono | kyoto | gases | copenhague
5,pt | aecio | minas | psdb | lula,pimentel | aecio | patrus | horizonte | minas
6,hospital | lula | ele | cirurgia | ontem,cirurgia | siriolibanes | quimioterapia | hosp...
7,presos | seguranca | presidios | justica | fed...,presidios | presos | penitenciaria | detentos ...
8,bilhoes | ano | fiscal | ministro | economia,levy | fiscal | ajuste | meta | despesas
9,estados | fiscal | lei | municipios | ano,lrf | estados | municipios | fiscal | divida
